# MO651 - Robótica Móvel - Trabalho 01
### Equipe: 
* Daiane Mendes - RA: 230210
* Carlos Victor - RA: 230261
* João Paulo - RA: 230221
* Pedro Olímpio - RA: 230256


## Introdução

Este trabalho tem como objetivo utilizar um robô em um ambiente simulado para explorar e mapear um cenário complexo. Além disso, usaremos técnicas de odometria para estimar a posição do robô durante a exploração do ambiente.

Usaremos o simulador v-rep e o robô Pioneer 3DX para execução dos experimentos. Com base nos resultados obtidos, realizaremos uma análise comparativa, verificando a qualidade dos mapeamentos feitos e das posições estimadas com odometria.

O cenário explorado conta com diversas sala conectadas por portas e diversos obstáculos, como cadeiras, mesas, armários, pessoes, entre outros.

O robô possui duas rodas independentes para locomoção e diversos sensores para obtenção de informações do ambiente, como uma câmera e sensores de proximidade.

O controle do robô será feito obtendo informações do ambiente através dos sensores, planejando as ações com base nessas informações e agindo no ambiente através de atuadores, no caso, as rodas.

<img src="imgs/fluxo.png">

## Metodologia

Descreveremo as técnicas usadas para exploração do ambiente e cálculo da odometria.

### Célula para o imports

In [0]:
import sys, time, random, math, cv2
import numpy as np
from matplotlib import pyplot as plt

sys.path.insert(0, '../src')
from robot import Robot

### Célula para as funções de processamento de imagens

Utilizamos essas funções para encontrar portas no cenário utilizando a câmera. O objetivo de encontrar as portas é descobrir onde estão as salas adjacentes da sala atual no cenário para então acessá-las e mapea-las.

A função *pre_processes_image* converte a imagem do formato fornecido pela câmera na simulação para um formato diferente. Utilizaremos essa imagem nesse novo formato para aplicar processamentos e funções da biblioteca *OpenCV*.

A função *apply_mask_by_color* aplica um filtro que remove da imagem os objetos com cor diferente da cor da porta. Note que alteramos o cenário do experimento para as portas possuirem uma cor única. Essa função auxilia na detecção da posição das portas.

A função *calculate_center* retorna a posição do contróide de uma porta na visão do robô. Caso não exista nenhuma porta na imagem obtida pela câmera, a função retorna uma tupla com valor falso no terceiro elemento. Caso existam mais de uma porta no campo de visão do robô, é retornado o centróide da porta de maior área na imagem. Utilizaremos essa função para encontrar portas.

In [0]:
def pre_process_image(resolution, image):
    img = np.array(image,dtype=np.uint8)
    img.resize([resolution[1],resolution[0],3])
    
    cv2.imwrite('vision'+'.png',cv2.flip(img, 0))
    
    return cv2.flip(img, 0)

def apply_mask_by_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Bitwise-AND mask and original image
    return cv2.bitwise_and(image,image, mask= mask)

def calculate_center(image):
    cv2.imwrite('masked'+'.png',image)
    # convert image to grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    # convert the grayscale image to binary image
    ret,thresh = cv2.threshold(gray_image,10,255, cv2.THRESH_BINARY)
    
    cv2.imwrite('thresh'+'.png',thresh)
    
    img, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    maximo = 0
    cmax = None
    for c in contours:
        area = cv2.contourArea(c)
        if area > maximo:
            maximo = area
            cmax = c
        
    # calculate moments for each contour
    M = cv2.moments(cmax)
    if(M["m00"] != 0 ):
        # calculate x,y coordinate of center
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        cv2.circle(image, (cX, cY), 5, (255, 255, 255), -1)
        cv2.putText(image, "center", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # display the image
        cv2.imwrite('door'+'.png',image)
        return cX, cY, True
    
    return 0, 0, False

### Célula para funções que mapeam o cenário

Utilizamos essas funções para encontrar e armazenar os obstáculos existentes no cenário.

A função *laser_sensors* utiliza os sensores de proximidade do robô para encontrar obstáculos próximos. Essa função também recebe como argumento a posição e orientação do robô e salva em uma imagem um mapa com a posição dos obstáculos encontrados.

A função *update_map* calcula a posição atual do robô, usando o *Ground Truth*, e as medições dos sensores e usa a função *laser_sensors* para atualizar o mapa dos obstáculos.

In [0]:
def laser_sensors(laser, c_position, gamma, label):
    x = []
    y = []
    
    rotation_matrix = np.matrix([[math.cos(gamma), -math.sin(gamma)], [math.sin(gamma), math.cos(gamma)]])
    for i in range(0, len(laser), 3):
        robot_position = np.matrix([[c_position[0]], [c_position[1]]])
        translation_matrix =  np.matrix([[laser[i]], [laser[i+1]]])
        T = np.matmul(rotation_matrix, translation_matrix)
        final_result = np.add(T, robot_position)
        x.append(final_result.item((0, 0)))
        y.append(final_result.item((1, 0)))
    plt.plot(x, y, label)

def update_map_gt():
    c_position = robot.get_current_position()
    c_orientation = robot.get_current_orientation()[2]
    laser = robot.read_laser()
    plt.figure(1)
    print(c_orientation)
    laser_sensors(laser, c_position, c_orientation, 'b.')
    plt.savefig('map_gt'+'.png')

def update_map_odometry(o_position, orientation):
    laser = robot.read_laser()
    plt.figure(2)
    print(orientation)
    laser_sensors(laser, o_position, orientation, 'r.')
    plt.savefig('map_odometry'+'.png')

def update_map(o_position, orientation):
    update_map_gt()
    update_map_odometry(o_position, orientation)

### Célula para funções que determinam o movimento do robô

Essas funções definem a maneira que o robô vai se movimentar pelo ambiente. As regras para evitar colisão, evitar ficar preso e decisão da direção que o robô seguirá são definidas a seguir.

A função *front_sensors_move* usa os 4 sensores frontais de proximidade para decidir a velocidade que será aplicada em cada roda do robô (esquerda e direita). Caso não existam obstáculos à frente do robô, a velocidade aplicada nas duas rodas será a mesma, ocasionando um movimento para frente. Caso exista algum obstáculo à frente do robô, será escolhida aleatóriamente uma direção que ele seguirá (esquerda ou direita). Caso exista algum obstáculo sendo detectado em alguma das direções, a direção que o robô seguirá não será aleatória, mas será aquela em que não exista obstáculo. Para isso, uma velocidade positiva será aplicada a uma das rodas e uma velocidade com valor oposto será aplicada na outra, ocasionando uma rotação do robô sobre o próprio eixo. Por fim, caso existam obstáculos na duas direções verificadas, aplicamos uma velocidade negativa em ambas as rodas, fazendo o robô se mover para trás.

<img src="imgs/algo.png">

A função *walk* recebe a velocidade de cada roda e o tempo em segundos que o robô deverá manter essa velocidade. A função também é responsável por manter o robô em movimento com a velocidade e o tempo passados por parâmetro.

Caso encontre uma porta, a função *enter_door* executa um procedimento para explorar a sala do outro lado da porta e sair. Para isso, quando uma porta é detectada pela câmera, o robô gira até estar alinhado com o centróide da porta. Em seguida, o robô se move em linha reta até ficar próximo à porta. Depois disso, o robô gira à direita até que seus sensores de proximidade frontais não detectem obstáculos próximos, ou seja, encontrou a passagem para a outra sala. Então, o robô avança para dentro da sala, vira à direita e à esquerda guardando as informações dos sensores sobre o ambiente e retorna de ré para a sala anterior.

Para essa função de entrar na porta alteramos o código robot.py para definir como 10 metros a distância máxima de detecção dos sensores.

In [0]:
def front_sensors_move(dist, vel):
    vLeft = vRight = defaultVelocity
    if (dist[3] < noDetectionDist or dist[4] < noDetectionDist):
        side = random.randint(0,1)
        if (dist[2] < noDetectionDistDiag and dist[5] > noDetectionDistDiag):
            side = 0
        elif (dist[5] < noDetectionDistDiag and dist[2] > noDetectionDistDiag):
            side = 1
        elif (dist[2] < noDetectionDistDiag and dist[5] < noDetectionDistDiag):
            distance = robot.read_ultrassonic_sensors()
            if (distance[2] < noDetectionDistDiag or distance[5] < noDetectionDistDiag or distance[1] < noDetectionDistDiag or distance[6] < noDetectionDistDiag):
                return[-defaultVelocity, -defaultVelocity]
        if (side != 0):
            distance = robot.read_ultrassonic_sensors()
            if (distance[3] < noDetectionDist or distance[4] < noDetectionDist):
                return [-(defaultVelocity)/2, defaultVelocity/2]
        else:
            distance = robot.read_ultrassonic_sensors()
            if (distance[3] < noDetectionDist or distance[4] < noDetectionDist):
                return [defaultVelocity/2, -defaultVelocity/2]
    return [vLeft, vRight]

def walk(left_velocity, right_velocity, time_to_walk):
    robot.set_left_velocity(left_velocity)
    robot.set_right_velocity(right_velocity)  
    time.sleep(time_to_walk)


def enter_door(position, orientation):
    resolution, image = robot.read_vision_sensor()
    aux = resolution[0]/2
    image = pre_process_image(resolution, image)
    door = apply_mask_by_color(image)
    X,Y,isCenter = calculate_center(door)
    X += 5
    
    if (isCenter):
        while isCenter and abs(X - aux) > 5:
            if(X < aux):
                robot.set_left_velocity(-0.05)
                robot.set_right_velocity(0.05)

            elif(X >= aux):
                robot.set_left_velocity(0.05)
                robot.set_right_velocity(-0.05)
                
            _, image = robot.read_vision_sensor()
            image = pre_process_image(resolution, image)
            door = apply_mask_by_color(image)
            X,Y,isCenter = calculate_center(door)
            X += 5
        if not isCenter:
            return
    
        us_distances = robot.read_ultrassonic_sensors()
        dist_front = min(us_distances[3], us_distances[4])
        while (dist_front > 0.3):
            robot.set_left_velocity(min(defaultVelocity, dist_front))
            robot.set_right_velocity(min(defaultVelocity, dist_front))
            us_distances = robot.read_ultrassonic_sensors()
            dist_front = min(us_distances[3], us_distances[4])
            
        while (dist_front < 7):
            robot.set_left_velocity(0.3)
            robot.set_right_velocity(-0.3)
            us_distances = robot.read_ultrassonic_sensors()
            dist_front = min(us_distances[3], us_distances[4])
        time.sleep(0.5)
        
        walk(defaultVelocity, defaultVelocity, 15)
        update_map(position, orientation)
        
        walk(1, -1, 4)
        update_map(position, orientation)

        walk(-1, 1, 8)
        update_map(position, orientation)
        
        walk(1, -1, 4)
        
        walk(-defaultVelocity, -defaultVelocity, 20)
        walk(2, -2, 2)


### Célula para o cálculo da Odometria

Para esses cálculos alteramos o código *robot.py*, para adicionar a função *get_joint_position*.

In [0]:
def calculate_delta(begin, end, spinOrient):
    if math.isclose(float(end), float(begin), abs_tol=0.00001):
        return 0
    end = (limit + end) % limit
    begin = (limit + begin) % limit

    ans = abs(end - begin)
    if spinOrient:
        if begin < end:
            return limit - ans
        return ans
    if begin > end:
        return limit - ans
    return ans

def calculate_speed(time, clockwiseSpin, lastEncoder, motorHandle):
    encoder = robot.get_joint_position(motorHandle)
    delta = calculate_delta(lastEncoder, encoder, clockwiseSpin)
    
    lastEncoder = encoder
    if delta > math.pi:
        delta = (2 * math.pi) - delta
        clockwiseSpin = not clockwiseSpin
    
    speed = 0.0975 * (delta/time)
    if(clockwiseSpin):
        return -speed, clockwiseSpin, lastEncoder
    
    return speed, clockwiseSpin, lastEncoder

def to180Universe(alpha):
    alpha = alpha % (2 * math.pi)
    if alpha > math.pi:
        return alpha - (2 * math.pi)
    return alpha

def to360Universe(alpha):
    if alpha < 0:
        return (2 * math.pi) + alpha
    return alpha

def add_angle(alpha, beta):
    angle = to360Universe(alpha) + to360Universe(beta)
    return to180Universe(angle)

def new_position(alpha, x, y):
    global lastTime
    global lastEncoderL, lastEncoderR
    global clockwiseSpinL, clockwiseSpinR
    
    now = time.time()
    dtime = now-lastTime
    lastTime = now
    
    vL, clockwiseSpinL, lastEncoderL = calculate_speed(dtime, clockwiseSpinL, lastEncoderL, 'left')
    vR, clockwiseSpinR, lastEncoderR = calculate_speed(dtime, clockwiseSpinR, lastEncoderR, 'right')
    
    w = (vR-vL)/0.36205
    v = (vR + vL)/2.0
    
    dAlpha = w*dtime
    dL = vL*dtime
    dR = vR*dtime
    dM = (dL+dR)/2.0
    dX = dM*math.cos(add_angle(alpha, dAlpha/2.0))
    dY = dM*math.sin(add_angle(alpha, dAlpha/2.0))
    return x+dX, y+dY, add_angle(alpha, dAlpha)

def plot_odometry_vs_GT(path, path_odometry):
    _, ax = plt.subplots(figsize=(9, 9))
    ax.plot(path_odometry[:,0], path_odometry[:,1], label = 'Odometry')
    ax.plot(path[:,0], path[:,1], label = 'Ground Truth')
    ax.grid(True)
    ax.legend(loc='best')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title('Odometry x Ground Truth')
    plt.savefig('odometry'+'.png') 
#     plt.show()

### Célula do laço principal de controle

Nesta célula, são definidas as variáveis que definirão como o robô irá se movimentar, ou seja, sua velocidade e sua distância de obstáculos. Isto é feito para planejar as ações do robô em diferentes circunstâncias.

Variáveis para o cálculo da odometria também são definidas nesta célula. As informações de *Ground Truth* serão utilizadas como posição e orientação iniciais do robô para o cálculo da odometria de iterações seguintes.

O laço principal será executado enquanto o robô estiver conectado. A cada iteração, usamos a função *front_sensors_move* para definir a velocidade que será aplicada em cada roda. Essas velocidades serão mantidas por 3 segundos. A cada iteração, a rotina de entrar em outra sala caso uma porta seja encontrada é chamada. Também a cada iteração é feito o cálculo de odometria para estimar a próxima posição.

In [0]:
noDetectionDist = 0.7
defaultVelocity = 1
noDetectionDistDiag = 0.4

robot = Robot()

limit = 2*math.pi
lastTime=time.time()
clockwiseSpinL = False
clockwiseSpinR = False
lastEncoderL = robot.get_joint_position('left')
lastEncoderR = robot.get_joint_position('right')

position = robot.get_current_position()
orientation = robot.get_current_orientation()

alpha = orientation[2]
x = position[0]
y = position[1]


path = []
path_odometry = []

while(robot.get_connection_status() != -1):
    
    position = robot.get_current_position()
    us_distances = robot.read_ultrassonic_sensors()
    vel = front_sensors_move(us_distances[:8], defaultVelocity)
    walk(vel[0], vel[1], 3)
    walk(0, 0, 0)
    
    update_map([x,y,0], alpha)
        
    enter_door([x,y,0], alpha)
    
    # Odometria
    x, y, alpha = new_position(alpha, x, y)
    path.append([-position[1], position[0]])
    path_odometry.append([-y, x])
    

Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
-0.0001970789598999545
-5.952484207227826e-05
-0.00034695438807830215
0.00013240725978747037
-0.00040907642687670887
-6.005046665524816e-05
-0.0006218625348992646
-0

###Célula para construir os gráficos de odometria

In [0]:
path = np.array(path)
path_odometry = np.array(path_odometry)

difx = path[1,0] - path_odometry[1,0]
dify = path[1,1] - path_odometry[1,1]
path_odometry[:,0] = path_odometry[:,0]+difx
path_odometry[:,1] = path_odometry[:,1]+dify

plot_odometry_vs_GT(path, path_odometry)

## Resultados

Mostraremos os resultados obtidos neste trabalho.


A imagem a seguir mostra o cenário em que os testes foram realizados.

<img src="imgs/scene.png">

Usamos as posições e orientações do *Ground Truth* para criar um mapa do ambiente. Nas figuras exibidas a seguir os pontos representas um obstáculo qualquer, seja ele uma parede, perna de uma cadeira ou um sofá.

Usando as técnicas descritas, de forma autônoma o robô mapeou grande parte do cenário. A imagem a seguir mostra o mapa obtido.

<img src="imgs/mapa_gt_so.png">

Alterando manualmente a posição do robô conseguimos o mapeamento da figura a seguir:

<img src="imgs/mapa_gt_mov.png">

Alterando apenas duas vezes a posição do robô conseguimos mapear quase por completo todo o cenário.

Geramos também mapas usando as posições estimadas do robô pela odometria e comparamos com os mapas gerados usando a posições do Ground Truth. As duas figuras a seguir mostram um mapa obtido usando as posições e orientações do Ground Truth (em azul) e da odometria (em vermelho).

<img src="imgs/map_gt.png">

<img src="imgs/map_odometry.png">

Esses mapas foram feitos apenas com um simples movimento do robô, apenas avançar alguns metros. Portanto, as posições da odometria estão muito próxima das exatas e o mapa está com erros pequenos. O gráfico a seguir mostra a rota real seguida pelo robô e a rota estimada pela odometria para esse experimento.

<img src="imgs/odometry1.png">

Com a inclusão de curvas no percurso as posições estimadas pela odometria começam a distanciar das posições reais. Na construção dos mapas obstáculos foram redesenhados em diferentes posições e orientações, gerando mapas incorretos. As duas imagens a seguir mostram mapas obtidos com as posições reais obtidas pelo Ground Truth (em azul) e posições estimadas da odometria (em vermelho).

<img src="imgs/map_gt2.png">

<img src="imgs/map_odometry2.png">

A seguir o gráfico que mostra as rotas calculadas usando o Ground Truth e odometria.

<img src="imgs/odometry2.png">

Apesar de distânciar da rota real, a rota calculada usando odometria ainda está muito semelhante a rota real, reproduzindo quase perfeitamente direção das curvas e comprimento das retas, falhando apenas em calcular o tamanho das curvas.

## Conclusões

Nesse trabalho conseguimos mapear o ambiente quase por completo, guardando localização dos obstáculos de quase todos o cenário. Além disso, usando técnicas de odometria, conseguimos calcular a posição do robô com erros pequenos em relação a posição real.